In [1]:
pip install netcdf4

     |████████████████████████████████| 4.7MB 8.0MB/s 
     |████████████████████████████████| 296kB 47.8MB/s 


In [2]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=bdd8376a0ed6bc0089748b6aad08b88ae7e0c08269641705dec5d72474d261c7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
import pandas as pd
import netCDF4
import numpy as np
import datetime
import time
from google.colab import files
import wget
import os

In [4]:
tile='h29v11'

### Load Unique date, Lon, Lat file to Pandas

In [5]:
df = pd.read_excel('https://drive.google.com/uc?id=1TCr_1liKtvDYhVSwKBfWah6jsIzTvzg5', header=0)
df.head()

,fih_year1,fih_date1,LAT,LON,MODIS
0,2001,2001-01-01,-34.4222,119.2147,h27v12
1,2001,2001-01-01,-34.2948,119.2456,h27v12
2,2001,2001-01-01,-33.5233,119.4422,h27v12
3,2001,2001-01-01,-33.6055,119.3786,h27v12
4,2001,2001-01-01,-34.4366,118.9687,h27v12


#### Initialise new columns

In [6]:
df['NC_LAT']=np.nan
df['NC_LON']=np.nan
df['lfmc_mean']=np.nan
df['lfmc_stdv']=np.nan
df['quality_mask']=np.nan
df.head()

,fih_year1,fih_date1,LAT,LON,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask
0,2001,2001-01-01,-34.4222,119.2147,h27v12,NaN,NaN,NaN,NaN,NaN
1,2001,2001-01-01,-34.2948,119.2456,h27v12,NaN,NaN,NaN,NaN,NaN
2,2001,2001-01-01,-33.5233,119.4422,h27v12,NaN,NaN,NaN,NaN,NaN
3,2001,2001-01-01,-33.6055,119.3786,h27v12,NaN,NaN,NaN,NaN,NaN
4,2001,2001-01-01,-34.4366,118.9687,h27v12,NaN,NaN,NaN,NaN,NaN


#h29v11 Tiles (2001-2020)

####Load h29v11 converted lat lon values
So we don't need to convert. 

In [7]:
wget.download('https://calwildfire.thesofttools.com/'+tile+'_lat_lon_ref.nc')

'h29v11_lat_lon_ref.nc'

In [8]:
!ls -ltrh

total 132M
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc


In [9]:
ll = netCDF4.Dataset(tile+'_lat_lon_ref.nc', mode='r')
lat = ll.variables['lat'][:][:]
lon = ll.variables['lon'][:][:]

#### loop through years

In [10]:
for year in range(2001,2021):
  print("Starting Year "+str(year)+"\n")
  print("============================\n")
  try:
    #Download netcdf file
    filename='fmc_c6_'+str(year)+'_'+tile+'.nc'
    wget.download('http://dapds00.nci.org.au/thredds/fileServer/ub8/au/FMC/c6/'+filename)
    nc = netCDF4.Dataset(filename, mode='r')
    !ls -ltrh
    #NC files having one record for every 4 days. Here we are making a array to refer the possible available date in nc file
    time_var = nc.variables['time'][:]
    dtime = netCDF4.num2date(time_var[:], 'seconds since 1970-01-01 00:00:00.0')
    dateidx = {}
    for i in range(dtime.shape[0]):
      dateidx[dtime[i].strftime("%Y%m%d")] = i
      # print dtime[i].strftime("%Y%m%d")
      for j in range(1, 4):
        t = dtime[i] + datetime.timedelta(days=j)
        dateidx[t.strftime("%Y%m%d")] = i
    
    #Get other values also to an array
    lfmc_mean = nc.variables['lfmc_mean'][:][:][:]
    lfmc_stdv = nc.variables['lfmc_stdv'][:][:][:]
    quality_mask = nc.variables['quality_mask'][:][:][:]
    for index, row in df.iterrows():
      if row['fih_date1']>=pd.to_datetime(str(year)+'0101', format='%Y%m%d', errors='ignore') and row['fih_date1']<=pd.to_datetime(str(year)+'1231', format='%Y%m%d', errors='ignore') and row['MODIS']==tile:
        diff = abs(lat - row['LAT']) + abs(lon - row['LON'])
        idx = np.unravel_index(diff.argmin(), diff.shape)
        try:
          did = dateidx[row['fih_date1'].strftime('%Y%m%d')]
          df.at[index, 'NC_LAT']=float(lat[idx[0]][idx[1]])
          df.at[index, 'NC_LON']=float(lon[idx[0]][idx[1]])
          df.at[index, 'lfmc_mean']=float(lfmc_mean[did][idx[0]][idx[1]])
          df.at[index, 'lfmc_stdv']=float(lfmc_stdv[did][idx[0]][idx[1]])
          df.at[index, 'quality_mask']=float(quality_mask[did][idx[0]][idx[1]])
        except Exception as e:
          print(df.loc[[index]])
          print(e)
    os.remove(filename)
  except Exception as e:
      print(e)
  print("End Year "+str(year)+"\n")
df.to_excel('WesternAustraliaFireDataUniqDateLatLonModis'+tile+'.xlsx',index=False)
files.download('/content/WesternAustraliaFireDataUniqDateLatLonModis'+tile+'.xlsx')

Starting Year 2001


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc
-rw-r--r-- 1 root root 4.4G Jan  4 05:16 fmc_c6_2001_h29v11.nc


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Warning: converting a masked element to nan.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Warning: converting a masked element to nan.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Warning: converting a masked element to nan.


End Year 2001

Starting Year 2002


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc
-rw-r--r-- 1 root root 4.5G Jan  4 05:23 fmc_c6_2002_h29v11.nc
End Year 2002

Starting Year 2003


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc
-rw-r--r-- 1 root root 4.5G Jan  4 05:31 fmc_c6_2003_h29v11.nc
End Year 2003

Starting Year 2004


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc
-rw-r--r-- 1 root root 4.5G Jan  4 05:41 fmc_c6_2004_h29v11.nc
End Year 2004

Starting Year 2005


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
-rw-r--r-- 1 root root 132M Jan  4 05:12 h29v11_lat_lon_ref.nc
-rw-r--r-- 1 root root 4.5G Jan  4 05:48 fmc_c6_2005_h29v11.nc
End Year 2005

Starting Year 2006


total 4.6G
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>